In [2]:
from google.colab import userdata

In [3]:
import requests
import pandas as pd

# Replace with your actual key or environment variable
CENSUS_API_KEY = userdata.get('CENSUS_USA')

# Extended set of demographic variables for bias analysis
# Variables are drawn from ACS 5-Year Profile
variables = {
    "NAME": "Census Tract Name",
    "DP05_0001E": "Total Population",
    "DP05_0038PE": "Black or African American (%)",
    "DP05_0071PE": "Hispanic or Latino (%)",
    "DP05_0077PE": "White Non-Hispanic (%)",
    "DP05_0033PE": "Female (%)",
    "DP05_0019PE": "Under 18 (%)",
    "DP02_0067PE": "High School or Less (%)",
    "DP02_0012PE": "Single Mother Households (%)",
    "DP03_0121PE": "Poverty Rate (%)",
    "DP03_0119PE": "Uninsured (%)",
    "DP04_0046PE": "Renter Occupied Housing (%)",
    "DP03_0121E": "People Below Poverty (Count)",
    "DP02_0012E": "Households with Children (Count)",
    "DP04_0059E": "Renter Occupied Units (Count)"
}
def API_request(year):
  # Construct the API URL
  url = (
      f"https://api.census.gov/data/{year}/acs/acs5/profile?"
      "get=" + ",".join(variables.keys()) +
      "&for=tract:*&in=state:48" +
      f"&key={CENSUS_API_KEY}"
  )
  print(url)
  # Request data from the Census API
  response = requests.get(url)

  # Validate the request was successful
  if response.status_code != 200:
      print("Error:", response.status_code, response.text)
      raise SystemExit("⛔ Failed to retrieve data from Census API")

  # Load JSON response into a DataFrame
  json_data = response.json()
  columns = json_data[0]
  rows = json_data[1:]
  df = pd.DataFrame(rows, columns=columns)

  # Convert numeric columns from string to appropriate type
  for col in list(variables.keys()):
      if col != "NAME":
          df[col] = pd.to_numeric(df[col], errors="coerce")

  # Generate GEOID for future joins with external datasets
  df["GEOID"] = df["state"] + df["county"] + df["tract"]

  # Rename columns using English-friendly labels
  df.rename(columns=variables, inplace=True)

  # Reorder columns for clarity
  ordered_columns = ["GEOID"] + list(variables.values()) + ["state", "county", "tract"]
  df = df[ordered_columns]
  return df

In [4]:
df1 = API_request(2023)
df1

https://api.census.gov/data/2023/acs/acs5/profile?get=NAME,DP05_0001E,DP05_0038PE,DP05_0071PE,DP05_0077PE,DP05_0033PE,DP05_0019PE,DP02_0067PE,DP02_0012PE,DP03_0121PE,DP03_0119PE,DP04_0046PE,DP03_0121E,DP02_0012E,DP04_0059E&for=tract:*&in=state:48&key=1da3c7110138e63071c4c789df242be0b25753ee


ConnectTimeout: HTTPSConnectionPool(host='api.census.gov', port=443): Max retries exceeded with url: /data/2023/acs/acs5/profile?get=NAME,DP05_0001E,DP05_0038PE,DP05_0071PE,DP05_0077PE,DP05_0033PE,DP05_0019PE,DP02_0067PE,DP02_0012PE,DP03_0121PE,DP03_0119PE,DP04_0046PE,DP03_0121E,DP02_0012E,DP04_0059E&for=tract:*&in=state:48&key=1da3c7110138e63071c4c789df242be0b25753ee (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7947a0638a90>, 'Connection to api.census.gov timed out. (connect timeout=None)'))

In [ ]:
df2 = API_request(2022)
df2

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48001950100,Census Tract 9501; Anderson County; Texas,5268,4.0,12.5,0.5,5268,22.5,91.3,9.1,55.2,14.0,86.1,-888888888,167,420,48,001,950100
1,48001950401,Census Tract 9504.01; Anderson County; Texas,4763,33.4,29.8,0.7,4763,0.0,68.4,0.0,-666666666.0,0.0,80.0,-888888888,0,2,48,001,950401
2,48001950402,Census Tract 9504.02; Anderson County; Texas,6540,37.1,16.9,1.9,6540,0.0,70.1,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-888888888,0,0,48,001,950402
3,48001950500,Census Tract 9505; Anderson County; Texas,4323,18.5,17.5,0.6,4323,27.7,88.5,15.1,34.9,13.8,66.9,-888888888,215,496,48,001,950500
4,48001950600,Census Tract 9506; Anderson County; Texas,5448,13.1,18.2,5.9,5448,28.8,86.9,23.1,0.0,22.2,57.5,-888888888,459,663,48,001,950600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,48505950402,Census Tract 9504.02; Zapata County; Texas,2396,0.5,46.0,1.3,2396,34.2,65.5,10.7,15.9,19.8,74.4,-888888888,95,258,48,505,950402
6892,48507950100,Census Tract 9501; Zavala County; Texas,1124,0.0,40.1,0.0,1124,38.2,70.1,18.0,25.0,37.6,74.6,-888888888,51,85,48,507,950100
6893,48507950200,Census Tract 9502; Zavala County; Texas,1161,5.3,39.9,0.0,1161,25.9,61.9,0.8,-666666666.0,25.6,77.5,-888888888,3,109,48,507,950200
6894,48507950301,Census Tract 9503.01; Zavala County; Texas,1960,0.0,44.5,1.8,1960,20.7,69.5,4.1,75.3,20.7,49.4,-888888888,36,280,48,507,950301


In [ ]:
df3 = API_request(2021)
df3

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48001950100,"Census Tract 9501, Anderson County, Texas",5447,3.9,11.8,76.0,5447,26.3,90.1,8.1,30.9,15.1,88.0,-888888888,148,416,48,001,950100
1,48001950401,"Census Tract 9504.01, Anderson County, Texas",4544,37.0,31.6,28.2,4544,1.1,68.9,0.0,-666666666.0,0.0,44.4,-888888888,0,7,48,001,950401
2,48001950402,"Census Tract 9504.02, Anderson County, Texas",6997,37.0,29.2,31.9,6997,0.0,71.2,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-888888888,0,0,48,001,950402
3,48001950500,"Census Tract 9505, Anderson County, Texas",4236,15.9,30.9,49.3,4236,28.7,88.7,14.8,43.1,13.3,65.4,-888888888,204,532,48,001,950500
4,48001950600,"Census Tract 9506, Anderson County, Texas",5843,32.5,17.4,50.0,5843,24.0,68.9,20.8,0.0,27.7,56.6,-888888888,399,699,48,001,950600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,48505950402,"Census Tract 9504.02, Zapata County, Texas",2620,0.0,95.6,4.4,2620,37.6,61.4,10.3,10.1,32.1,72.4,-888888888,90,264,48,505,950402
6892,48507950100,"Census Tract 9501, Zavala County, Texas",1090,0.0,98.2,1.8,1090,37.3,71.7,18.4,0.0,26.2,68.8,-888888888,50,87,48,507,950100
6893,48507950200,"Census Tract 9502, Zavala County, Texas",1014,4.3,90.4,5.2,1014,31.9,62.4,0.7,0.0,28.3,68.2,-888888888,2,122,48,507,950200
6894,48507950301,"Census Tract 9503.01, Zavala County, Texas",1882,0.0,82.5,16.5,1882,19.9,68.5,4.5,80.0,14.0,45.7,-888888888,35,187,48,507,950301


In [ ]:
df4 = API_request(2020)
df4

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48029181004,"Census Tract 1810.04, Bexar County, Texas",6669,9.8,41.4,34.3,6669,19.4,92.8,20.0,28.9,12.9,8.4,-888888888,564,1780,48,029,181004
1,48029181005,"Census Tract 1810.05, Bexar County, Texas",4000,18.7,52.7,18.0,4000,26.5,90.3,18.1,20.8,22.5,6.7,-888888888,326,1003,48,029,181005
2,48029181100,"Census Tract 1811, Bexar County, Texas",6152,0.6,46.5,43.3,6152,21.6,92.7,24.2,0.0,3.0,73.9,-888888888,595,1057,48,029,181100
3,48029181200,"Census Tract 1812, Bexar County, Texas",5350,1.7,42.0,51.2,5350,12.2,95.7,21.8,0.0,0.0,81.4,-888888888,463,673,48,029,181200
4,48029181301,"Census Tract 1813.01, Bexar County, Texas",4894,5.1,55.1,28.2,4894,18.4,88.8,17.0,13.2,6.0,30.1,-888888888,348,1059,48,029,181301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,48453002409,"Census Tract 24.09, Travis County, Texas",3365,4.0,40.8,53.5,3365,13.3,90.3,15.6,0.0,8.5,70.5,-888888888,224,384,48,453,002409
6892,48453002410,"Census Tract 24.10, Travis County, Texas",3980,3.7,45.1,43.7,3980,17.9,86.7,15.2,0.0,15.9,28.8,-888888888,272,920,48,453,002410
6893,48453002411,"Census Tract 24.11, Travis County, Texas",6316,8.7,85.5,6.1,6316,27.7,59.1,5.1,24.0,18.3,48.3,-888888888,83,321,48,453,002411
6894,48453002412,"Census Tract 24.12, Travis County, Texas",5310,4.7,84.6,10.2,5310,26.7,58.6,3.3,34.5,21.9,63.1,-888888888,44,317,48,453,002412


In [ ]:
df5 = API_request(2019)
df5

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48061013500,"Census Tract 135, Cameron County, Texas",1773,0.5,83.4,15.2,1773,20.8,73.6,15.4,0.0,16.0,74.5,-888888888,95,196,48,061,013500
1,48061013305,"Census Tract 133.05, Cameron County, Texas",4708,0.0,99.1,0.9,4708,28.1,58.5,8.4,0.0,23.3,67.4,-888888888,112,369,48,061,013305
2,48061013309,"Census Tract 133.09, Cameron County, Texas",2849,0.0,99.7,0.3,2849,35.9,44.8,4.1,30.8,33.2,61.6,-888888888,29,262,48,061,013309
3,48061013402,"Census Tract 134.02, Cameron County, Texas",1999,0.3,96.1,3.8,1999,31.4,48.6,12.3,22.7,35.4,59.3,-888888888,69,227,48,061,013402
4,48061012613,"Census Tract 126.13, Cameron County, Texas",5286,4.3,88.4,4.1,5286,23.1,85.6,13.8,0.0,5.9,76.3,-888888888,238,482,48,061,012613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48329010114,"Census Tract 101.14, Midland County, Texas",11351,4.4,38.0,51.1,11351,29.7,93.7,17.9,3.3,6.3,62.8,-888888888,765,1508,48,329,010114
5261,48329010200,"Census Tract 102, Midland County, Texas",4360,13.7,80.0,7.7,4360,34.1,66.9,6.3,54.9,5.7,79.0,-888888888,72,508,48,329,010200
5262,48329980000,"Census Tract 9800, Midland County, Texas",0,-666666666.0,-666666666.0,-666666666.0,0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-888888888,0,0,48,329,980000
5263,48331950401,"Census Tract 9504.01, Milam County, Texas",3262,14.4,20.1,64.3,3262,27.6,79.4,11.8,29.2,15.9,68.5,-888888888,151,461,48,331,950401


In [ ]:
df6 = API_request(2018)
df6

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48135002200,"Census Tract 22, Ector County, Texas",3288,0.4,74.5,24.9,3288,29.6,8.9,3.4,0.0,12.8,83.9,-888888888.0,37,320,48,135,002200
1,48061012800,"Census Tract 128, Cameron County, Texas",5345,0.2,97.0,3.0,5345,28.1,9.1,11.7,0.0,19.5,73.3,-888888888.0,178,490,48,061,012800
2,48061013204,"Census Tract 132.04, Cameron County, Texas",2240,0.0,95.1,3.9,2240,33.1,6.9,11.8,41.2,30.6,66.9,-888888888.0,75,271,48,061,013204
3,48061013307,"Census Tract 133.07, Cameron County, Texas",1782,0.0,100.0,0.0,1782,29.7,6.5,10.4,81.8,35.7,65.0,-888888888.0,53,175,48,061,013307
4,48061013902,"Census Tract 139.02, Cameron County, Texas",4197,0.5,100.0,0.0,4197,29.9,13.4,5.8,67.9,34.9,56.1,-888888888.0,64,440,48,061,013902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48213950902,"Census Tract 9509.02, Henderson County, Texas",2144,18.4,4.3,74.4,2144,20.6,19.1,17.9,0.0,20.8,68.9,-888888888.0,156,332,48,213,950902
5261,48213950903,"Census Tract 9509.03, Henderson County, Texas",2261,1.5,2.3,92.4,2261,20.1,19.3,16.1,-666666666.0,7.9,86.0,-888888888.0,166,375,48,213,950903
5262,48215020101,"Census Tract 201.01, Hidalgo County, Texas",6820,0.1,96.6,3.4,6820,38.5,4.1,12.0,43.0,51.7,50.1,-888888888.0,225,969,48,215,020101
5263,48215020102,"Census Tract 201.02, Hidalgo County, Texas",8546,0.0,79.5,20.5,8546,26.6,14.2,10.0,0.0,11.1,84.6,-888888888.0,275,1377,48,215,020102


In [ ]:
df7 = API_request(2017)
df7

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48479001815,"Census Tract 18.15, Webb County, Texas",1479,0.0,98.3,1.3,1479,31.0,7.0,3.4,43.1,46.8,61.8,-888888888.0,15,141,48,479,001815
1,48479001812,"Census Tract 18.12, Webb County, Texas",9224,0.0,95.1,4.6,9224,40.8,14.0,0.8,0.0,24.9,79.8,-888888888.0,15,423,48,479,001812
2,48479001813,"Census Tract 18.13, Webb County, Texas",831,0.0,94.2,5.8,831,31.8,8.7,11.6,-666666666.0,32.3,76.4,-888888888.0,28,71,48,479,001813
3,48479001814,"Census Tract 18.14, Webb County, Texas",7915,0.0,99.2,0.4,7915,39.4,9.8,1.6,0.0,29.2,76.0,-888888888.0,31,510,48,479,001814
4,48479001816,"Census Tract 18.16, Webb County, Texas",4757,0.9,96.8,2.3,4757,37.1,9.0,2.3,-666666666.0,20.5,91.7,-888888888.0,25,298,48,479,001816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48085031712,"Census Tract 317.12, Collin County, Texas",5557,22.3,17.1,42.4,5557,20.1,62.6,7.3,0.0,2.2,29.7,-888888888.0,200,1490,48,085,031712
5261,48085031713,"Census Tract 317.13, Collin County, Texas",4147,30.6,16.4,35.6,4147,18.0,41.9,2.7,27.8,13.7,15.5,-888888888.0,60,1415,48,085,031713
5262,48085031714,"Census Tract 317.14, Collin County, Texas",6390,33.0,23.7,30.0,6390,22.1,44.0,4.7,20.7,16.8,5.2,-888888888.0,155,2060,48,085,031714
5263,48029131506,"Census Tract 1315.06, Bexar County, Texas",3541,19.6,54.8,17.6,3541,29.7,17.1,4.1,0.0,6.9,63.3,-888888888.0,47,346,48,029,131506


In [ ]:
df8 = API_request(2016)
df8

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48113011300,"Census Tract 113, Dallas County, Texas",4985,0.0,93.8,0.0,90.4,77.7,17.7,20.7,30.5,22.1,65.5,-888888888,399,1002,48,113,011300
1,48113012209,"Census Tract 122.09, Dallas County, Texas",3459,0.0,56.7,0.0,19.6,65.4,41.1,9.4,15.6,15.9,49.7,-888888888,114,292,48,113,012209
2,48113012800,"Census Tract 128, Dallas County, Texas",7520,0.0,65.6,0.0,7.2,77.3,37.5,11.6,0.0,10.8,73.9,-888888888,363,890,48,113,012800
3,48113013725,"Census Tract 137.25, Dallas County, Texas",4405,0.0,46.6,0.0,13.3,67.6,30.2,6.2,28.1,20.2,7.7,-888888888,117,971,48,113,013725
4,48113013804,"Census Tract 138.04, Dallas County, Texas",3428,0.0,55.1,0.0,5.3,71.6,36.8,16.7,0.0,7.6,74.8,-888888888,214,403,48,113,013804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48121021613,"Census Tract 216.13, Denton County, Texas",5660,0.0,64.2,0.0,16.6,63.8,27.8,2.5,17.3,18.4,32.7,-888888888,53,992,48,121,021613
5261,48121020401,"Census Tract 204.01, Denton County, Texas",8347,0.0,76.8,0.0,6.1,71.8,29.3,2.7,29.8,8.5,56.0,-888888888,76,714,48,121,020401
5262,48121021614,"Census Tract 216.14, Denton County, Texas",3998,0.0,52.9,0.0,4.4,70.9,24.4,9.0,6.2,6.2,76.5,-888888888,119,377,48,121,021614
5263,48121020305,"Census Tract 203.05, Denton County, Texas",7480,0.0,94.5,0.0,3.6,75.3,55.5,4.0,0.0,2.5,92.2,-888888888,108,462,48,121,020305


In [ ]:
df9 = API_request(2015)
df9

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48439105902,"Census Tract 1059.02, Tarrant County, Texas",6404,0.0,76.1,0.0,28.3,71.9,13.6,7.7,76.9,43.7,13.5,-888888888,102,569,48,439,105902
1,48439110807,"Census Tract 1108.07, Tarrant County, Texas",6542,0.0,70.6,0.4,15.6,68.6,29.0,4.4,13.8,11.0,71.2,-888888888,100,481,48,439,110807
2,48439106517,"Census Tract 1065.17, Tarrant County, Texas",8500,0.0,90.1,0.0,43.2,68.8,29.4,1.5,24.6,12.6,37.5,-888888888,50,1773,48,439,106517
3,48439106518,"Census Tract 1065.18, Tarrant County, Texas",4104,0.0,84.7,0.5,34.7,83.6,43.7,1.9,26.4,9.3,0.0,-888888888,46,1570,48,439,106518
4,48439110600,"Census Tract 1106, Tarrant County, Texas",2584,0.0,73.5,0.2,1.9,74.3,30.3,11.1,7.5,5.5,50.3,-888888888,119,497,48,439,110600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48189950200,"Census Tract 9502, Hale County, Texas",3952,0.0,22.2,0.5,8.5,61.3,3.3,8.6,23.3,32.5,51.3,-888888888,105,416,48,189,950200
5261,48189950400,"Census Tract 9504, Hale County, Texas",4856,0.0,39.8,0.0,6.2,62.7,17.6,4.4,20.3,10.4,45.5,-888888888,66,523,48,189,950400
5262,48419950200,"Census Tract 9502, Shelby County, Texas",3564,0.0,89.1,0.0,13.9,73.4,17.7,8.5,42.0,21.3,72.5,-888888888,119,527,48,419,950200
5263,48419950600,"Census Tract 9506, Shelby County, Texas",2921,0.0,93.2,0.0,21.9,78.2,17.2,12.9,0.0,15.1,77.8,-888888888,155,275,48,419,950600


In [ ]:
df10 = API_request(2014)
df10

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract
0,48013960100,"Census Tract 9601, Atascosa County, Texas",7075,0.0,49.9,0.0,0.8,68.8,21.5,8.8,0.0,8.5,32.4,-888888888,213,787,48,013,960100
1,48013960201,"Census Tract 9602.01, Atascosa County, Texas",7164,0.0,35.5,0.0,0.4,65.0,9.7,6.4,13.2,10.7,26.9,-888888888,141,927,48,013,960201
2,48013960300,"Census Tract 9603, Atascosa County, Texas",3763,0.0,8.1,0.0,0.0,62.6,3.3,7.5,66.7,24.6,28.6,-888888888,88,385,48,013,960300
3,48013960402,"Census Tract 9604.02, Atascosa County, Texas",7849,0.0,48.2,0.3,1.2,67.7,21.7,10.8,22.6,12.6,34.1,-888888888,281,1040,48,013,960402
4,48013960401,"Census Tract 9604.01, Atascosa County, Texas",2136,0.0,23.2,0.0,0.0,70.1,8.1,6.8,0.0,20.3,13.7,-888888888,52,307,48,013,960401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48461950100,"Census Tract 9501, Upton County, Texas",1127,0.0,71.2,0.0,1.1,72.1,10.3,7.2,9.4,5.0,34.7,-888888888,29,183,48,461,950100
5261,48021950100,"Census Tract 9501, Bastrop County, Texas",9633,0.0,69.5,2.1,1.5,71.2,17.1,3.9,0.0,11.4,15.3,-888888888,127,1348,48,021,950100
5262,48065950100,"Census Tract 9501, Carson County, Texas",2348,0.0,94.5,0.0,0.1,72.5,22.0,11.3,21.6,4.7,12.0,-888888888,111,429,48,065,950100
5263,48065950200,"Census Tract 9502, Carson County, Texas",3760,0.0,88.4,0.0,1.5,69.1,23.0,8.6,0.0,3.4,11.0,-888888888,116,518,48,065,950200


In [ ]:
# Concatene the dataframes
dict_df = {2023: df1,
           2022: df2,
           2021: df3,
           2020: df4,
           2019: df5,
           2018: df6,
           2017: df7,
           2016: df8,
           2015: df9,
           2014: df10}
for year,df in dict_df.items():
  df["year"] = year
df_final = pd.concat(dict_df.values())
df_final

,GEOID,Census Tract Name,Total Population,Black or African American (%),Hispanic or Latino (%),White Non-Hispanic (%),Female (%),Under 18 (%),High School or Less (%),Single Mother Households (%),Poverty Rate (%),Uninsured (%),Renter Occupied Housing (%),People Below Poverty (Count),Households with Children (Count),Renter Occupied Units (Count),state,county,tract,year
0,48001950100,Census Tract 9501; Anderson County; Texas,5359,6.0,4.2,8.0,5359.0,22.9,92.2,10.2,26.0,16.3,86.1,-888888888.0,196,411,48,001,950100,2023
1,48001950401,Census Tract 9504.01; Anderson County; Texas,4196,34.6,3.6,31.3,4196.0,0.0,67.1,0.0,-666666666.0,0.0,65.4,-888888888.0,0,2,48,001,950401,2023
2,48001950402,Census Tract 9504.02; Anderson County; Texas,5848,34.8,2.1,30.8,5848.0,0.0,69.9,-666666666.0,-666666666.0,-666666666.0,-666666666.0,-888888888.0,0,0,48,001,950402,2023
3,48001950500,Census Tract 9505; Anderson County; Texas,4544,17.8,0.6,31.5,4544.0,24.8,88.1,13.3,57.5,14.0,70.9,-888888888.0,190,472,48,001,950500,2023
4,48001950600,Census Tract 9506; Anderson County; Texas,5292,14.7,0.5,24.2,5292.0,24.0,86.9,15.7,0.0,22.7,57.3,-888888888.0,306,632,48,001,950600,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,48461950100,"Census Tract 9501, Upton County, Texas",1127,0.0,71.2,0.0,1.1,72.1,10.3,7.2,9.4,5.0,34.7,-888888888.0,29,183,48,461,950100,2014
5261,48021950100,"Census Tract 9501, Bastrop County, Texas",9633,0.0,69.5,2.1,1.5,71.2,17.1,3.9,0.0,11.4,15.3,-888888888.0,127,1348,48,021,950100,2014
5262,48065950100,"Census Tract 9501, Carson County, Texas",2348,0.0,94.5,0.0,0.1,72.5,22.0,11.3,21.6,4.7,12.0,-888888888.0,111,429,48,065,950100,2014
5263,48065950200,"Census Tract 9502, Carson County, Texas",3760,0.0,88.4,0.0,1.5,69.1,23.0,8.6,0.0,3.4,11.0,-888888888.0,116,518,48,065,950200,2014


In [ ]:
df_final.to_csv("census_data.csv", index=False)